<a href="https://colab.research.google.com/github/sasansharifipour/Spark_Class/blob/main/Spark_Schema.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("pySpark Sql basic example")\
        .getOrCreate()

from pyspark.sql import Row
sc = spark.sparkContext

In [3]:
lines = sc.textFile('/content/spark-3.0.1-bin-hadoop2.7/examples/src/main/resources/people.txt')

In [4]:
parts = lines.map(lambda line : line.split(","))

people = parts.map(lambda p : Row(name=p[0], age=int(p[1])))

In [5]:
schemaPeople = spark.createDataFrame(people)
schemaPeople.createTempView("people")

In [6]:
teenagers = spark.sql("SELECT name FROM people WHERE age >= 13 and age <= 19")

In [7]:
teenagers

DataFrame[name: string]

In [8]:
teenagers.show()

+------+
|  name|
+------+
|Justin|
+------+



In [9]:
teenNames = teenagers.rdd.map(lambda p: 'Name:' + p.name)

In [10]:
teenNames.collect()

['Name:Justin']

In [11]:
from pyspark.sql.types import *

In [12]:
lines2 = sc.textFile('/content/spark-3.0.1-bin-hadoop2.7/examples/src/main/resources/people.txt')

In [13]:
parts2 = lines2.map(lambda line : line.split(","))

In [14]:
people2 = parts2.map(lambda p : (p[0], p[1].strip()))

In [16]:
schemaString = "name age"

In [18]:
fields2 = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema2 = StructType(fields2)

In [19]:
schema2

StructType(List(StructField(name,StringType,true),StructField(age,StringType,true)))

In [20]:
schemaPeople2 = spark.createDataFrame(people2, schema2)

schemaPeople2.createTempView("persons")

In [21]:
results = spark.sql("SELECT name FROM persons")

In [22]:
results.show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



In [23]:
spark.stop()
sc.stop()